## Scalability Testing

This notebook is for testing purposes.

In [8]:
import random

from config import Color
from helpers import clean
from mavlink.customtypes.location import ENUPose, GRAPose
from plan import Plan
from simulator import (
    QGC,
    ConfigGazebo,
    ConfigNovis,
    ConfigQGC,
    Gazebo,
    NoVisualizer,
    Simulator,
)

clean()


## Simulation Configuration

In [9]:
gra_origin = GRAPose(lat=-35.3633280, lon=149.1652241,alt=0,heading=90) 
enu_origin = ENUPose(x=0, y=0, z=gra_origin.alt, heading=gra_origin.heading) 

gcs_colors=[
    Color.RED,
    Color.ORANGE,
    Color.GREEN,
    Color.BLUE
    ]

n_uavs_per_gcs = 60
side_len = 10
altitude = 5
max_delay = 3 #sec

base_homes = ENUPose.list([
    (i * 50, j * 3 * side_len, 0, 0)
    for i in range(len(gcs_colors))
    for j in range(n_uavs_per_gcs)
])

base_paths = [
    Plan.create_square_path(side_len=side_len, alt=altitude,heading=0) 
    for _ in base_homes
    ]

colors = [color for color in gcs_colors for _ in range(n_uavs_per_gcs)]

msn_delays=[random.randint(0, max_delay) for _ in base_homes]


## Assign vehicles to GCS (by color)
gcs_sysids = {f"{color.name}_{color.emoji}":
    list(range(i * n_uavs_per_gcs + 1, (i + 1) * n_uavs_per_gcs + 1))
    for i, color in enumerate(gcs_colors)
    }

## Gazebo Configuration

In [10]:
gaz_config = ConfigGazebo(origin = enu_origin,
                        world_path="simulator/gazebo/worlds/runway3.world")

for path,home,c in zip(base_paths,base_homes,colors):
    gaz_config.add(base_path=path,base_home=home,color=c)
#gaz_config.show()

gaz= Gazebo(gaz_config,gra_origin)

## QGroundControl Configuration

In [11]:
qgc_config = ConfigQGC(origin = gra_origin)

for path,home,color,delay in zip(base_paths,base_homes,colors,msn_delays):
    qgc_config.add(base_path=path,base_home=home,color=color,mission_delay=delay)
qgc_config.show()

qgc=QGC(qgc_config)

# No Visualizer

In [12]:
novis_config = ConfigNovis(origin=gra_origin)
for home in base_homes:
    novis_config.add(base_home=home)
novis=NoVisualizer(novis_config)

## Launch Simulator

In [13]:
simulator = Simulator(
	visualizers=[novis],
	gcs_system_ids=gcs_sysids,
	missions=[veh.mission for veh in qgc_config.vehicles],
	terminals=['gcs'],
	verbose=1,
)

orac = simulator.launch()

01:28:49 - Oracle ⚪ - INFO - 🙈 Running without visualization.
01:28:49 - Oracle ⚪ - INFO - 🚀 GCS RED_🟥 launched (PID 1203403)
01:28:49 - Oracle ⚪ - INFO - 🚀 GCS ORANGE_🟧 launched (PID 1203404)
01:28:49 - Oracle ⚪ - INFO - 🚀 GCS GREEN_🟩 launched (PID 1203405)
01:28:49 - Oracle ⚪ - INFO - 🚀 GCS BLUE_🟦 launched (PID 1203406)
01:28:49 - Oracle ⚪ - INFO - 🔗 Starting Oracle connections to vehicles...


01:29:41 - Oracle ⚪ - INFO - 🔗 UAV logic 1 is connected to Oracle ⚪
01:29:42 - Oracle ⚪ - INFO - 🔗 UAV logic 21 is connected to Oracle ⚪
01:29:42 - Oracle ⚪ - INFO - 🔗 UAV logic 2 is connected to Oracle ⚪
01:29:42 - Oracle ⚪ - INFO - 🔗 UAV logic 15 is connected to Oracle ⚪
01:29:42 - Oracle ⚪ - INFO - 🔗 UAV logic 8 is connected to Oracle ⚪
01:29:42 - Oracle ⚪ - INFO - 🔗 UAV logic 3 is connected to Oracle ⚪
01:29:42 - Oracle ⚪ - INFO - 🔗 UAV logic 7 is connected to Oracle ⚪
01:29:42 - Oracle ⚪ - INFO - 🔗 UAV logic 5 is connected to Oracle ⚪
01:29:42 - Oracle ⚪ - INFO - 🔗 UAV logic 10 is connected to Oracle ⚪
01:29:42 - Oracle ⚪ - INFO - 🔗 UAV logic 6 is connected to Oracle ⚪
01:29:42 - Oracle ⚪ - INFO - 🔗 UAV logic 13 is connected to Oracle ⚪
01:29:42 - Oracle ⚪ - INFO - 🔗 UAV logic 11 is connected to Oracle ⚪
01:29:42 - Oracle ⚪ - INFO - 🔗 UAV logic 22 is connected to Oracle ⚪
01:29:42 - Oracle ⚪ - INFO - 🔗 UAV logic 9 is connected to Oracle ⚪
01:29:42 - Oracle ⚪ - INFO - 🔗 UAV logic 3

## Oracle checking

In [14]:
orac.run()

01:29:52 - Oracle ⚪ - INFO - 🏁 Starting Oracle with 240 vehicles and 4 GCSs
01:31:12 - Oracle ⚪ - INFO - Received message 'DONE' from GCS BLUE_🟦
01:31:12 - Oracle ⚪ - INFO - GCS BLUE_🟦 removal complete. Remaining connections: 180
01:31:13 - Oracle ⚪ - INFO - Received message 'DONE' from GCS GREEN_🟩
01:31:13 - Oracle ⚪ - INFO - GCS GREEN_🟩 removal complete. Remaining connections: 120
01:31:14 - Oracle ⚪ - INFO - Received message 'DONE' from GCS ORANGE_🟧
01:31:14 - Oracle ⚪ - INFO - GCS ORANGE_🟧 removal complete. Remaining connections: 60
01:31:14 - Oracle ⚪ - INFO - Received message 'DONE' from GCS RED_🟥
01:31:14 - Oracle ⚪ - INFO - GCS RED_🟥 removal complete. Remaining connections: 0
01:31:14 - Oracle ⚪ - INFO - ✅ Main monitoring loop completed - all connections closed
01:31:14 - Oracle ⚪ - INFO - 🎉 Oracle shutdown complete!
